In [1]:
import pandas as pd
import zipfile
import geopandas as gpd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
z = zipfile.ZipFile('../data_initial/mappluto_14v2.zip')
z.extractall('../data_initial/')

In [3]:
#boroughs = ['Staten_Island', 'Queens', 'Manhattan', 'Bronx', 'Brooklyn']
#abreevs = ['SI', 'QN', 'MN', 'BX', 'BK']

boroughs = ['Staten_Island']
abreevs = ['SI']

pluto = []
i = 0
for elem in boroughs:
    temp = gpd.read_file('../data_initial/'+elem+'/'+abreevs[i]+'MapPLUTO.shp')
    temp['BBL'] = temp['BBL'].astype(str)
    temp = temp.to_crs({'init':'epsg:4326'})
    pluto.append(temp)
    i+=1

In [4]:
z = zipfile.ZipFile('../data_initial/cb_2014_36_bg_500k.zip')
z.extractall('../data_initial/')

In [5]:
cbg = gpd.read_file('../data_initial/cb_2014_36_bg_500k.shp')

In [6]:
cbg = cbg.to_crs({'init':'epsg:4326'})

In [7]:
cbg = cbg[['geometry','GEOID']]
cbg['GEOID'] = cbg['GEOID'].astype(str)

In [8]:
abreevsl = [elem.lower() for elem in abreevs]
bbl_cbg = []
missing_bbls = []

for i in range(0,len(pluto)):
    for_merge = pluto[i][['BBL','geometry']]
    for_merge['BBL'] = for_merge['BBL'].astype(str)
    merged = gpd.sjoin(for_merge, cbg)
    merged = merged.drop(['geometry','index_right'],axis=1)
    bbl_cbg.append(merged)
    missing_bbls.append(list(set([elem for elem in pluto[i].BBL]) - set([elem for elem in merged.BBL])))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [9]:
bbl_cbg[0].head()

,BBL,GEOID
0,5007470028.0,360850189012
80,5007510245.0,360850189012
325,5004770074.0,360850189012
379,5004790001.0,360850189012
404,5004760035.0,360850189012


Specific features needed from PLUTO are;
* SUM(BldgArea)
* SUM(ResArea)
* SUM(UnitRes)
* SUM(ResArea) of non city-owned or federal-owned lots ("OwnerType" is not 'C' or 'O')
* SUM(UnitsRes) of non city-owned or federal-owned lots ("OwnerType" is not 'C' or 'O')

In [10]:
bbl_cbg[0].dtypes

BBL      object
GEOID    object
dtype: object

In [11]:
pluto[0].head(2)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,None,437 PURDY AVENUE,R3-1,None,7,15120.0,23400.0,5007470028.0,1488,...,0,1960,None,10314,R3-1,None,None,None,20d,POLYGON ((-74.14250944979312 40.60980609846242...
1,0.0,None,CUNARD AVENUE,R3A/HS,None,4,21518.0,21518.0,5006230252.0,0,...,0,0,None,10304,R3A,None,None,None,21d,POLYGON ((-74.08780465394567 40.61401983466899...


In [13]:
pluto[0].ResArea.unique()

array([ 1488,     0,  1316, ..., 77068, 12786, 93672])

In [61]:
bbl_cbg[0].head(2)

,BBL,GEOID
0,5007470028.0,360850189012
80,5007510245.0,360850189012


In [62]:
bldg_area = []
for i in range(0,len(pluto)):
    bldgarea = pd.merge(bbl_cbg[i], pluto[i], on='BBL')
    bldgarea = pd.DataFrame(bldgarea['BldgArea'].groupby(bldgarea['GEOID']).sum())
    bldgarea = bldgarea.reset_index()
    bldg_area.append(bldgarea)

In [63]:
res_area = []
for i in range(0,len(pluto)):
    resarea = pd.merge(bbl_cbg[i], pluto[i], on='BBL')
    resarea = pd.DataFrame(resarea['ResArea'].groupby(resarea['GEOID']).sum())
    resarea = resarea.reset_index()
    res_area.append(resarea)

In [64]:
units_res = []
for i in range(0,len(pluto)):
    unitsres = pd.merge(bbl_cbg[i], pluto[i], on='BBL')
    unitsres = pd.DataFrame(unitsres['UnitsRes'].groupby(unitsres['GEOID']).sum())
    unitsres = unitsres.reset_index()
    units_res.append(unitsres)

In [65]:
res_area_non_govt = []
for i in range(0,len(pluto)):
    resarea = pd.merge(bbl_cbg[i], pluto[i], on='BBL')
    resarea['OwnerType'].astype(str)
    resarea = resarea[(resarea['OwnerType'] != 'O') & (resarea['OwnerType'] != 'C')]
    resarea = pd.DataFrame(resarea['ResArea'].groupby(resarea['GEOID']).sum())
    resarea = resarea.rename(columns={'ResArea':'ResAreaNonGovt'})
    resarea = resarea.reset_index()
    res_area_non_govt.append(resarea)

In [66]:
res_units_non_govt = []
for i in range(0,len(pluto)):
    resunits = pd.merge(bbl_cbg[i], pluto[i], on='BBL')
    resunits['OwnerType'].astype(str)
    resunits = resunits[(resunits['OwnerType'] != 'O') & (resunits['OwnerType'] != 'C')]
    resunits = pd.DataFrame(resunits['UnitsRes'].groupby(resunits['GEOID']).sum())
    resunits = resunits.rename(columns={'UnitsRes':'UnitsResNonGovt'})
    resunits = resunits.reset_index()
    res_units_non_govt.append(resunits)

In [67]:
total_assessed = []
for i in range(0,len(pluto)):
    assessed = pd.merge(bbl_cbg[i], pluto[i], on='BBL')
    assessed = pd.DataFrame(assessed['AssessTot'].groupby(assessed['GEOID']).mean())
    assessed = assessed.reset_index()
    total_assessed.append(assessed)

In [68]:
pluto[3]['Borough'].unique()

array([u'BX'], dtype=object)

In [69]:
#concat each list to create one long list of all boroughs in NYC
bldg_area = pd.concat(bldg_area)
res_area = pd.concat(res_area)
units_res = pd.concat(units_res)
res_area_non_govt = pd.concat(res_area_non_govt)
res_units_non_govt = pd.concat(res_units_non_govt)
total_assessed = pd.concat(total_assessed)

In [70]:
#merge features from disparate dfs to one on GEOID
features = pd.merge(bldg_area, cbg, on='GEOID')

In [71]:
features.head(2)

,GEOID,BldgArea,geometry
0,360850003001,1504448,"POLYGON Z ((-74.079206 40.643431 0, -74.074681..."
1,360850003002,2761678,"POLYGON Z ((-74.078198 40.642115 0, -74.077899..."


In [72]:
features = pd.merge(res_area, features, on='GEOID')

In [73]:
features.head(2)

,GEOID,ResArea,BldgArea,geometry
0,360850003001,371603,1504448,"POLYGON Z ((-74.079206 40.643431 0, -74.074681..."
1,360850003002,779368,2761678,"POLYGON Z ((-74.078198 40.642115 0, -74.077899..."


In [74]:
features = pd.merge(units_res, features, on='GEOID')

In [75]:
features.head(2)

,GEOID,UnitsRes,ResArea,BldgArea,geometry
0,360850003001,432,371603,1504448,"POLYGON Z ((-74.079206 40.643431 0, -74.074681..."
1,360850003002,694,779368,2761678,"POLYGON Z ((-74.078198 40.642115 0, -74.077899..."


In [76]:
features = pd.merge(res_area_non_govt, features, on='GEOID')

In [77]:
features.head()

,GEOID,ResAreaNonGovt,UnitsRes,ResArea,BldgArea,geometry
0,360850003001,371603,432,371603,1504448,"POLYGON Z ((-74.079206 40.643431 0, -74.074681..."
1,360850003002,779368,694,779368,2761678,"POLYGON Z ((-74.078198 40.642115 0, -74.077899..."
2,360850006001,288227,253,288882,1361189,"POLYGON Z ((-74.075647 40.626664 0, -74.068037..."
3,360850006002,962290,1076,1183380,1488942,POLYGON Z ((-74.06613399999999 40.614089000000...
4,360850007001,118923,601,559638,617093,POLYGON Z ((-74.0899307181427 40.6479466397106...


In [78]:
features = pd.merge(res_units_non_govt, features, on='GEOID')

In [79]:
features.head(2)

,GEOID,UnitsResNonGovt,ResAreaNonGovt,UnitsRes,ResArea,BldgArea,geometry
0,360850003001,432,371603,432,371603,1504448,"POLYGON Z ((-74.079206 40.643431 0, -74.074681..."
1,360850003002,694,779368,694,779368,2761678,"POLYGON Z ((-74.078198 40.642115 0, -74.077899..."


In [80]:
features = pd.merge(total_assessed, features, on='GEOID')

In [81]:
features.head()

,GEOID,AssessTot,UnitsResNonGovt,ResAreaNonGovt,UnitsRes,ResArea,BldgArea,geometry
0,360850003001,2.288829e+06,432,371603,432,371603,1504448,"POLYGON Z ((-74.079206 40.643431 0, -74.074681..."
1,360850003002,8.011592e+05,694,779368,694,779368,2761678,"POLYGON Z ((-74.078198 40.642115 0, -74.077899..."
2,360850006001,2.494912e+05,252,288227,253,288882,1361189,"POLYGON Z ((-74.075647 40.626664 0, -74.068037..."
3,360850006002,2.016866e+05,798,962290,1076,1183380,1488942,POLYGON Z ((-74.06613399999999 40.614089000000...
4,360850007001,1.267398e+05,99,118923,601,559638,617093,POLYGON Z ((-74.0899307181427 40.6479466397106...


In [82]:
len(features)

8955

In [83]:
features.isnull().sum()

GEOID              0
AssessTot          0
UnitsResNonGovt    0
ResAreaNonGovt     0
UnitsRes           0
ResArea            0
BldgArea           0
geometry           0
dtype: int64

In [84]:
type(features)

pandas.core.frame.DataFrame